In [1]:
!pip install opendatasets pandas -q


In [2]:
import opendatasets as od
import pandas
#{"username":"khwrali","key":"a2b23dfbac2443ab4db34e48318ce4ff"}
od.download(
    "https://www.kaggle.com/datasets/sripaadsrinivasan/audio-mnist")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: khwrali
Your Kaggle Key: ··········


100%|██████████| 948M/948M [00:09<00:00, 106MB/s]


In [3]:
import os
import librosa
import numpy as np
from tqdm import tqdm

In [4]:
data_folder = '/content/audio-mnist/data'

def extract_mfccs(file_path, num_mfcc=13):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc)
    return mfccs


In [5]:
num_mfcc = 13
num_instances = 50
num_speakers = 60
num_digits = 10

In [6]:
fixed_length = 100

mfccs_data = []
labels = []

for speaker_id in tqdm(range(1, num_speakers + 1), desc='Speakers'):
    speaker_folder = os.path.join(data_folder, f'{speaker_id:02d}')

    for digit in range(num_digits):
        for instance in tqdm(range(num_instances), desc=f'Digit {digit}'):
            file_name = f'{digit}_{speaker_id:02d}_{instance}.wav'
            file_path = os.path.join(speaker_folder, file_name)
            mfccs = extract_mfccs(file_path, num_mfcc)
            if mfccs.shape[1] < fixed_length:
                mfccs = np.pad(mfccs, ((0, 0), (0, fixed_length - mfccs.shape[1])))
            else:
                mfccs = mfccs[:, :fixed_length]

            mfccs_data.append(mfccs)
            labels.append(digit)

Streaming output truncated to the last 5000 lines.
Digit 0: 100%|██████████| 50/50 [00:00<00:00, 56.08it/s]

Digit 1: 100%|██████████| 50/50 [00:00<00:00, 74.75it/s]

Digit 2: 100%|██████████| 50/50 [00:00<00:00, 83.90it/s]

Digit 3: 100%|██████████| 50/50 [00:00<00:00, 73.87it/s]

Digit 4: 100%|██████████| 50/50 [00:00<00:00, 80.02it/s]

Digit 5: 100%|██████████| 50/50 [00:00<00:00, 78.04it/s]

Digit 6: 100%|██████████| 50/50 [00:00<00:00, 67.07it/s]

Digit 7: 100%|██████████| 50/50 [00:00<00:00, 61.33it/s]

Digit 8: 100%|██████████| 50/50 [00:00<00:00, 85.29it/s]

Digit 0: 100%|██████████| 50/50 [00:00<00:00, 68.82it/s]

Digit 1: 100%|██████████| 50/50 [00:00<00:00, 80.26it/s]

Digit 2: 100%|██████████| 50/50 [00:00<00:00, 72.69it/s]

Digit 3: 100%|██████████| 50/50 [00:00<00:00, 65.85it/s]

Digit 4: 100%|██████████| 50/50 [00:00<00:00, 53.40it/s]

Digit 5: 100%|██████████| 50/50 [00:01<00:00, 49.52it/s]

Digit 6: 100%|██████████| 50/50 [00:01<00:00, 39.41it/s]

Digit 7: 100%|███████

In [7]:
mfccs_data = np.array(mfccs_data)
labels = np.array(labels)
print("MFCCs shape:", mfccs_data.shape)
print("Labels shape:", labels.shape)

MFCCs shape: (30000, 13, 100)
Labels shape: (30000,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    mfccs_data, labels, test_size=0.1, random_state=42, stratify=labels
)

print("Train set shapes:", X_train.shape, y_train.shape)
print("Test set shapes:", X_test.shape, y_test.shape)


Train set shapes: (27000, 13, 100) (27000,)
Test set shapes: (3000, 13, 100) (3000,)


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(num_mfcc, fixed_length)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)


Epoch 1/10
844/844 [==============================] - 90s 96ms/step - loss: 0.2707 - accuracy: 0.9209 - val_loss: 0.1097 - val_accuracy: 0.9640
Epoch 2/10
844/844 [==============================] - 82s 97ms/step - loss: 0.0727 - accuracy: 0.9784 - val_loss: 0.0581 - val_accuracy: 0.9833
Epoch 3/10
844/844 [==============================] - 81s 96ms/step - loss: 0.0441 - accuracy: 0.9871 - val_loss: 0.0478 - val_accuracy: 0.9870
Epoch 4/10
844/844 [==============================] - 79s 94ms/step - loss: 0.0342 - accuracy: 0.9899 - val_loss: 0.0778 - val_accuracy: 0.9757
Epoch 5/10
844/844 [==============================] - 80s 94ms/step - loss: 0.0346 - accuracy: 0.9894 - val_loss: 0.0393 - val_accuracy: 0.9910
Epoch 6/10
844/844 [==============================] - 81s 96ms/step - loss: 0.0287 - accuracy: 0.9904 - val_loss: 0.0444 - val_accuracy: 0.9873
Epoch 7/10
844/844 [==============================] - 80s 94ms/step - loss: 0.0220 - accuracy: 0.9936 - val_loss: 0.0400 - val_accuracy:

In [21]:
predictions = model.predict(X_test)


94/94 [==============================] - 5s 33ms/step


In [22]:
n=2000
print(y_test[n])
print(np.argmax(predictions[n]))

5
5
